# Dataset: *ebola-essien*

- df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-df.csv',index=False)
- src_tw_df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-src.csv',index=False)

In [2]:
import numpy as np
import pandas as pd
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-df-2.csv', encoding = 'utf-8', header = 0)
# df.dropna(inplace = True)
# df.reset_index(drop=True,inplace=True)
src_tw_df = pd.read_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-src.csv', encoding = 'utf-8', header =0)

## Emotion Analysis

### T5 Based
<https://huggingface.co/mrm8488/t5-base-finetuned-emotion?text=I+wish+you+were+here+but+it+is+impossible>

In [1]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSequenceClassification

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [4]:
t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion",use_fast=False)
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")



def get_emotion_t5(text):
    input_ids = t5_tokenizer.encode(text , return_tensors='pt')
    output = t5_model.generate(input_ids=input_ids,max_length=2)
  
    dec = [t5_tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\transformers\models\auto\modeling_auto.py:694: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
emotion_t5 = df.reply_tweet.apply(get_emotion_t5)

In [ ]:
df['reply_emo_t5'] = emotion_t5

In [ ]:
ax = df.reply_emotion.value_counts().plot.bar()

for i in ax.patches:
    height = i.get_height()
    ax.text(
        i.get_x() + i.get_width() / 2,height, str(height), ha="center", va="bottom"
    )
plt.xticks(rotation=0)
plt.show()
#plt.savefig('D:/論文/PHEME9/Code/graph/ebola-essien/rep-emotion.pdf',dpi=300)

### roBERTa-base model
<https://huggingface.co/cardiffnlp/twitter-roberta-base-emotion>

In [5]:
import numpy as np
from scipy.special import softmax

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
ro_tokenizer = AutoTokenizer.from_pretrained(MODEL)
ro_model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [6]:
def get_emotion_roBERT(text):
    
    def preprocess(txt):
        new_text = []
        for t in txt.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        return " ".join(new_text)
    
    labels = ['anger','joy','optimism','sadness']
    text = preprocess(text)
    encoded_input = ro_tokenizer(text, return_tensors='pt')
    output = ro_model(**encoded_input)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    emo = labels[ranking[0]]
    return emo

In [14]:
emotion_ro = df.reply_tweet.apply(get_emotion_roBERT)

In [15]:
df['reply_emo_ro'] = emotion_ro
df.to_csv(r'D:\論文\PHEME9\Data\CSV\ebola-essien-df-2.csv',index=False)

 - robert : 'anger','joy','optimism','sadness'
 - t5: 'sadness', 'joy', 'love', 'anger', 'fear', 'surprise' 
 - text2emotion: 'angry, 'fear', 'happy, 'sad', 'surprise'

In [13]:
print('roBERT based: ',get_emotion_roBERT(df.reply_tweet[2]))
print('t5 based: ',get_emotion_t5(df.reply_tweet[2]))
print('lexicon based: ', df.reply_emotion[2]) #text2emotion

roBERT based:  anger
t5 based:  <pad> sadness
lexicon based:  Fear


In [15]:
print('roBERT based: ',get_emotion_roBERT('wtf'))
print('t5 based: ',get_emotion_t5('wtf'))
print('lexicon based: ', df.reply_emotion[2])

roBERT based:  anger
t5 based:  <pad> fear
lexicon based:  Fear


In [10]:
print("reply tweet: ", df.reply_tweet[2])
print("cleaned reply tweet: ", df.cleaned_reply_tw[2])

reply tweet:  “@YuryAlkaev: Breaking news: Ghana international and AC Milan star Michael Essien has contracted Ebola, his club has confirmed.” Wtf
cleaned reply tweet:  yuryalkaev break news ghana intern ac milan star michael essien ha contract ebola hi club ha confirm wtf
